In [181]:
import numpy as np
import pandas as pd
from gensim import corpora, models, similarities
from gensim.models import word2vec

from sklearn import datasets
from sklearn import linear_model
from sklearn import svm
from sklearn import preprocessing
import random
import pickle
import time

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
base_path = "/home/R2016hwang/research/"
model_name_en = base_path + "word2vec/model_sg_en_wzh.w2v"
model_name_jp = base_path + "word2vec/model_sg_jp_wzh.w2v"
model_name_zh = base_path + "word2vec/model_sg_zh_wzh.w2v"

model_en = word2vec.Word2Vec.load(model_name_en)
model_jp = word2vec.Word2Vec.load(model_name_jp)
model_zh = word2vec.Word2Vec.load(model_name_zh)

trans_jp_en = np.load("word2vec/jp_en.npy")
trans_zh_en = np.load("word2vec/zh_en.npy")

datapath = base_path + "data_prepare/cleaned_jp_en_zh.csv"

2018-05-08 00:01:56,430 : INFO : loading Word2Vec object from /home/R2016hwang/research/word2vec/model_sg_en_wzh.w2v
2018-05-08 00:01:57,040 : INFO : loading wv recursively from /home/R2016hwang/research/word2vec/model_sg_en_wzh.w2v.wv.* with mmap=None
2018-05-08 00:01:57,041 : INFO : setting ignored attribute syn0norm to None
2018-05-08 00:01:57,043 : INFO : setting ignored attribute cum_table to None
2018-05-08 00:01:57,044 : INFO : loaded /home/R2016hwang/research/word2vec/model_sg_en_wzh.w2v
2018-05-08 00:01:57,176 : INFO : loading Word2Vec object from /home/R2016hwang/research/word2vec/model_sg_jp_wzh.w2v
2018-05-08 00:01:57,509 : INFO : loading wv recursively from /home/R2016hwang/research/word2vec/model_sg_jp_wzh.w2v.wv.* with mmap=None
2018-05-08 00:01:57,510 : INFO : setting ignored attribute syn0norm to None
2018-05-08 00:01:57,511 : INFO : setting ignored attribute cum_table to None
2018-05-08 00:01:57,512 : INFO : loaded /home/R2016hwang/research/word2vec/model_sg_jp_wzh.w2

In [3]:
maxlen = 20 # Default: 0 -> infinite
epoch = 10
dim_lstm = 100
dim_1 = 800
# dim_2 = 100
# dim_3 = 50
dropout_rate = 0.0
bias_y = 0
loss_function = "mse"
mode = "reg" # reg, binary
rnn_type = "bi-lstm" # lstm, bi-lstm
bi_lstm_mode = "sum" #concat, sum
print("maxlen", maxlen, "epoch", epoch, "dim_lstm", dim_lstm)
print("dim_Dense", dim_1)
print("dropout_rate", dropout_rate, ", LSTM type:", rnn_type, bi_lstm_mode)
p_activation = ["relu", "relu", "relu"]
print("Activation function:", p_activation)
print("bias of y:", bias_y)
print("loss_function:", loss_function)
start = 0
step = 10
print("start:", start, "end:", )
print("------------------------------")
random.seed(1234)

maxlen 20 epoch 10 dim_lstm 100
dim_Dense 800
dropout_rate 0.0 , LSTM type: bi-lstm sum
Activation function: ['relu', 'relu', 'relu']
bias of y: 0
loss_function: mse
start: 0 end:
------------------------------


In [225]:
torch.manual_seed(1)    # reproducible

# Hyper Parameters
EPOCH = 1          # 训练整批数据多少次
BATCH_SIZE = 100
TIME_STEP = 8      # rnn 时间步数 / 图片高度
INPUT_SIZE = 100     # rnn 每步输入值 / 图片每行像素
HIDDEN_SIZE = 100
BIDIRECTION = False
LR = 0.01           # learning rate
DROP_OUT = 0.0

In [226]:
class LaLSTM(nn.Module):
    def __init__(self, batch_size=BATCH_SIZE, time_step=TIME_STEP, hidden_size=HIDDEN_SIZE, input_size=INPUT_SIZE, dropout_p=DROP_OUT, bidirection=False, attention=False):
        super(LaLSTM, self).__init__()
        self.attention = attention
        self.lstm_left = nn.LSTM(     # LSTM 效果要比 nn.RNN() 好多了
            input_size=input_size,      # 图片每行的数据像素点
            hidden_size=hidden_size,     # rnn hidden unit
            num_layers=1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
            bidirectional=bidirection,
        )
        
        self.lstm_right = nn.LSTM(     # LSTM 效果要比 nn.RNN() 好多了
            input_size=input_size,      # 图片每行的数据像素点
            hidden_size=hidden_size,     # rnn hidden unit
            num_layers=1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
            bidirectional=bidirection,
        )
        
        if bidirection:
            print("bidirection")
            self.full_connect = torch.nn.Sequential(
                                                #torch.nn.Linear(hidden_size*2, hidden_size),
                                                #torch.nn.ReLU(),
                                                torch.nn.Linear(hidden_size*4, 1),
                                                torch.nn.ReLU()
                                                )
            self.attn_left = nn.Linear(4*hidden_size, hidden_size, bias=False)
            #self.attn_apply_left = nn.Linear(self.)
            self.attn_right = nn.Linear(4*hidden_size, hidden_size, bias=False)
        else:
            self.attn_left = nn.Linear(2*hidden_size, hidden_size, bias=False)
            #self.attn_apply_left = nn.Linear(self.)
            self.attn_right = nn.Linear(2*hidden_size, hidden_size, bias=False)

            self.full_connect = torch.nn.Sequential(
                                                    #torch.nn.Linear(hidden_size*2, hidden_size),
                                                    #torch.nn.ReLU(),
                                                    torch.nn.Linear(hidden_size*2, 1),
                                                    torch.nn.Sigmoid()
                                                    )

        self.dropout = nn.Dropout(dropout_p)
        #self.gru = nn.GRU(self.hidden_size, self.hidden_size)

        nn.init.xavier_uniform(self.conv1.weight)
    def forward(self, x_left, x_right):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)   LSTM 有两个 hidden states, h_n 是分线, h_c 是主线
        # h_c shape (n_layers, batch, hidden_size)
        # batch_first = True : (batch, time_step, input_size)
        
        ##left_out, (h_n_left, h_c_left) = self.lstm_left(x, None)   # None 表示 hidden state 会用全0的 state
        ##right_out, (h_n_right, h_c_right) = self.lstm_right(x, None)

        # 选取最后一个时间点的 r_out 输出
        # 这里 r_out[:, -1, :] 的值也是 h_n 的值
        ##out = self.out(r_out[:, -1, :])
        ##return out
    
        left_out, left_hidden = self.lstm_left(x_left)   # None 表示 hidden state 会用全0的 state
        right_out, right_hidden = self.lstm_right(x_right)
        #r_out_reshaped = r_out.view(-1, HIDDEN_SIZE) # to 2D data
        #outs = self.linear_layer(r_out_reshaped)
        #left_out = left_out.view(-1, TIME_STEP, INPUT_SIZE)  # to 3D data
        #right_out = right_out.view(-1, TIME_STEP, INPUT_SIZE)  # to 3D data
        
        #h_state_left = (h_n_left, h_c_left)
        #h_state_right = (h_n_right, h_c_right)
        
        # attention = h_left dot h_right.T
        #(batch, h_left, h_right)
        attn_matrix_left = torch.bmm(left_out,torch.transpose(right_out,1,2))
        #(batch, h_right, h_left)
        attn_matrix_right = torch.bmm(right_out,torch.transpose(left_out,1,2))
        left_at = F.softmax(attn_matrix_left,2)
        right_at = F.softmax(attn_matrix_right,2)
        left_ct = torch.bmm(left_at,right_out)
        right_ct = torch.bmm(right_at,left_out)
        #take only last attention for results without feed_in
        left_hn = left_out[:, -1, :].squeeze(1)
        right_hn = right_out[:, -1, :].squeeze(1)
        left_cn = left_ct[:, -1, :].squeeze(1)
        right_cn = right_ct[:, -1, :].squeeze(1)
        
        left_new_hn = F.tanh(self.attn_left(torch.cat((left_hn,left_cn),1)))
        right_new_hn = F.tanh(self.attn_right(torch.cat((right_hn,right_cn),1)))
        
        if self.attention:
            self.out = self.full_connect(torch.cat((left_new_hn,right_new_hn),1))
        else:
            self.out = self.full_connect(torch.cat((left_hn,right_hn),1))
        
        return self.out

lalstm = LaLSTM()
print(lalstm)

LaLSTM(
  (lstm_left): LSTM(100, 100, batch_first=True)
  (lstm_right): LSTM(100, 100, batch_first=True)
  (attn_left): Linear(in_features=200, out_features=100, bias=False)
  (attn_right): Linear(in_features=200, out_features=100, bias=False)
  (full_connect): Sequential(
    (0): Linear(in_features=200, out_features=1, bias=True)
    (1): Sigmoid()
  )
  (dropout): Dropout(p=0.0)
)


In [227]:
def prepare_train(dir_en_jp, second_language="jp", start=None, end=None):

    # df_en_mapping = pd.read_csv(dir_en)
    # df_jp_mapping = pd.read_csv(dir_jp)
    second_article = second_language+"_article"
    
    df_en_jp = pd.read_csv(dir_en_jp,
                              names=["HEADLINE_ALERT_TEXT_x","HEADLINE_ALERT_TEXT_y", "HEADLINE_ALERT_TEXT","jp_article","en_article","zh_article"],
                              header=0)
    df_en_mapping = df_en_jp[["en_article"]].iloc[start:end]
    df_jp_mapping = df_en_jp[[second_article]].iloc[start:end]

    print("Reading English Data:", len(df_en_mapping))
    print("Reading "+second_language+" Data:", len(df_jp_mapping))

    sample_size = len(df_en_mapping)

    assert len(df_en_mapping) == len(df_jp_mapping)

    # Convert mapping to list type and then concat to the a list
    print("Merging the English and Japanes news dataframe...")
    df_train_1 = pd.concat([df_en_mapping, df_jp_mapping], axis = 1)
    df_train_1['similarity'] = pd.Series(np.ones(sample_size,)*5)
    df_train_1['dis_similarity'] = pd.Series(np.ones(sample_size,)*1)

    # Remove null line
    print("Drop the null line...")
    # df_train_1 = df_train_1.dropna(subset=['en_article'])
    df_train_1 = df_train_1[df_train_1['en_article'] != '<NULL>']

    # Expand the training data
    en_article_wrong = df_train_1.en_article.iloc[random.sample(range(len(df_train_1)),len(df_train_1))]
    en_article_wrong.index = df_train_1.index
    print((en_article_wrong == df_train_1.en_article).value_counts())
    df_train_1['en_article_wrong'] = en_article_wrong

    # Convert dateframe to list
    train_1 = df_train_1[['en_article',second_article,'similarity']].values.tolist()
    train_2 = df_train_1[['en_article_wrong',second_article,'dis_similarity']].values.tolist()

    return train_1, train_2, df_train_1

In [228]:
def padding(sequence, maxlen=maxlen, padding_value=0.0):
    np_sequance = np.array(sequence)
    #print(np_sequance.shape)
    if np_sequance.shape[0] == 0:
        #return float('nan')
        print("长度为零")
        #return float('nan')
        return np.zeros((maxlen, INPUT_SIZE))
    if np_sequance.shape[0] < maxlen:
        z = np.zeros((maxlen, INPUT_SIZE))
    #    print(z.shape)
        z[:np_sequance.shape[0], :np_sequance.shape[1]] = np_sequance
    else:
        z = np_sequance[:maxlen, :]
    return z


In [229]:
def find_ranking_batch(projection1, projection2, dlmodel, batch=10):
    sim_results = []
    rank_results = []
    sample_length = len(projection2)

    # Iterate each of the ariticle from projection1 (999) as proj1
    # Calculate the simialrity of proj1 with all ariticles in projection2 (999)
    # for i, proj1 in enumerate(projection1):
    for i in range(0, sample_length, batch):
        print("Find answer for doc.", i, i+batch)
        proj1 = projection1[i:i+batch]

        proj1_tile = np.repeat(proj1, sample_length, axis=0)
        proj2_tile = np.tile(projection2, (batch,1,1))
        print(proj1_tile.shape)
        print(proj2_tile.shape)

        # For each batch, we should tile each of the element
        sim = dlmodel.predict([proj1_tile, proj2_tile])[:,0]
        for j in range(0, sample_length, batch):
            rank = pd.Series(sim[j:j+sample_length]).rank(ascending = False)[i]
            sim_results.append(sim)
            rank_results.append(rank)

    # sim_results contains 999*999 similairty matrix
    return sim_results, rank_results

def find_ranking(projection1, projection2, dlmodel):
    sim_results = []
    rank_results = []

    # Iterate each of the ariticle from projection1 (999) as proj1
    # Calculate the simialrity of proj1 with all ariticles in projection2 (999)
    for i, proj1 in enumerate(projection1):
        print("Find answer for doc.", i)
        proj1_tile = np.tile(proj1, (len(projection2), 1, 1))
        sim = dlmodel.predict([proj1_tile, projection2])[:,0]
        rank = pd.Series(sim).rank(ascending = False)[i]
        sim_results.append(sim)
        rank_results.append(rank)

    # sim_results contains 999*999 similairty matrix
    return sim_results, rank_results


In [230]:
def find_top(rank_results, top):
    s = pd.Series(rank_results)
    n_top = (s <= top).sum()
    return n_top


def average_docment(document_embedding):
    return np.average(document_embedding, axis=0)

def sum_docment(document_embedding):
    return np.sum(document_embedding, axis=0)

In [231]:
def doc2embed(doc,model,translation_matrix=None):
    # r = [model_en[token] for token in doc.split()]
    r = []
    r_failed = []

    
    for token in str(doc).split(' '):
        if token in model:
            if type(translation_matrix) != type(None) :
                #print("translation_matrix ",translation_matrix.shape)
                #print(np.array(model[token]).shape)
                r.append(np.array(model[token]).dot(translation_matrix))
            else:
                r.append(model[token])
        else:
            r_failed.append(token)

    # if len(r_failed) != 0:
    #     print " ".join(r_failed)
    #print("Failed Number",len(r_failed))
    return r


In [232]:
pairs_correct_enjp, pairs_wrong_enjp, df_pairs_enjp = prepare_train(datapath,second_language='jp')

Reading English Data: 81283
Reading jp Data: 81283
Merging the English and Japanes news dataframe...
Drop the null line...
False    80969
True       314
Name: en_article, dtype: int64


In [233]:
df_pairs_enjp

,en_article,jp_article,similarity,dis_similarity,en_article_wrong
0,china slow mid-term prospect bright-fitch,中国 経済成長 やや 減速 中期 見通し 良好 フィッチ,5.0,1.0,scenarios-hot topic market finance meeting
1,US stock open low ahead greenspan remark,米国 株式市場 序盤 小幅安 グリーンスパン 議長 証言 控え,5.0,1.0,instant view 6-job growth slow expect jan
2,US 3-month bill high rate pct,表 米 落札 結果 最高 落札 金利 3か月 6か月,5.0,1.0,poll-us aug retail sale seEUp pct
3,US 3-month bill high rate pct,表 米 落札 結果 最高 落札 金利 3か月 6か月,5.0,1.0,bank canada cut rate cite downside risk
4,US 3-month bill high rate pct,表 米 落札 結果 最高 落札 金利 3か月 6か月,5.0,1.0,stock snapshot-wall st jump ge rating outlook ...
5,US stock open slightly high ahead fed,米国 株式市場 序盤 小 反発 FOMC 結果 見守る,5.0,1.0,gm say significant interest asset sale
6,rpt-US stock open slightly high ahead fed,米国 株式市場 序盤 小 反発 FOMC 結果 見守る,5.0,1.0,imf say split eu ecb greece debt
7,blair say worry oil want low price,原油価格 問題 議題 なる 可能性 英 首相,5.0,1.0,factbox-electoral vote tally US election
8,s korea fin min see gdp 4-5 pct report,今年 下期 韓国 GDP 成長率 見通し 財政 経済,5.0,1.0,poll-boe cut again feb rate trough
9,s korea fin min see gdp 4-5 pct report,再送 今年 下期 韓国 GDP 成長率 見通し 財政 経済,5.0,1.0,factbox-electoral vote tally US election


In [234]:
pairs_correct_enzh, pairs_wrong_enzh, df_pairs_enzh = prepare_train(datapath,second_language='zh')

Reading English Data: 81283
Reading zh Data: 81283
Merging the English and Japanes news dataframe...
Drop the null line...
False    80955
True       328
Name: en_article, dtype: int64


In [235]:
df_pairs_enzh

,en_article,zh_article,similarity,dis_similarity,en_article_wrong
0,china slow mid-term prospect bright-fitch,重發 中國 經濟 惠譽 預計 GDP 增速 今明兩年 放緩 至 與 但 中期 前景 看好,5.0,1.0,eu euro long-term rate repeat forex
1,US stock open low ahead greenspan remark,美國股市 微幅 開 低 市場 等待 格老 證詞 演說,5.0,1.0,wrapup mccain face judgment US voter
2,US 3-month bill high rate pct,美國 債市 三個 月期 國庫券 標售 得標 利率 為 投標 倍數 為,5.0,1.0,german prelim feb cpi +0 pct m/m +0 pct y/y
3,US 3-month bill high rate pct,美國 債市 三個 月期 國庫券 標售 得標 利率 為 投標 倍數 為,5.0,1.0,stocks-indexes open high cisco data lift optimism
4,US 3-month bill high rate pct,美國 債市 三個 月期 國庫券 標售 得標 利率 為 投標 倍數 為,5.0,1.0,factbox-electoral vote tally US election
5,US stock open slightly high ahead fed,美國股市 開盤 微幅 走高 市場 等待 FED 利率 聲明,5.0,1.0,US mortgage application rise last week-mba
6,rpt-US stock open slightly high ahead fed,美國股市 開盤 微幅 走高 市場 等待 FED 利率 聲明,5.0,1.0,seoul share gain foreign buying brokerage
7,blair say worry oil want low price,國際 油市 布萊爾 稱 G 領導人 擔憂 油價 希望 各國 為 抑制 油價 作出努力,5.0,1.0,hk blue chip suffer worst loss year
8,s korea fin min see gdp 4-5 pct report,韓國 經濟 財長 預期 下半年 GDP 較 上年 同期 成長 報導,5.0,1.0,breakingviews-america corn monster bEUnstoppa...
9,s korea fin min see gdp 4-5 pct report,韓國 經濟 財長 預期 下半年 GDP 較 上年 同期 成長 報導,5.0,1.0,singapore april foreign reservEUp pct march


In [236]:
print("Using the new test data to evaluate.......")
df_pairs_evaluate = df_pairs_enjp.iloc[50000:55000:5]

df_pairs_evaluate['word2vec_en'] = df_pairs_evaluate['en_article'].apply(doc2embed,args=(model_en,))
df_pairs_evaluate['word2vec_en_wrong'] = df_pairs_evaluate['en_article_wrong'].apply(doc2embed,args=(model_en,))
df_pairs_evaluate['word2vec_jp'] = df_pairs_evaluate['jp_article'].apply(doc2embed,args=(model_jp,trans_jp_en))


df_pairs_evaluate['padding_en'] = df_pairs_evaluate['word2vec_en'].apply(padding, args=(TIME_STEP,))
df_pairs_evaluate['padding_en_wrong'] = df_pairs_evaluate['word2vec_en_wrong'].apply(padding, args=(TIME_STEP,))
df_pairs_evaluate['padding_jp'] = df_pairs_evaluate['word2vec_jp'].apply(padding, args=(TIME_STEP,))

df_pairs_evaluate.dropna(axis=0, how='all')


features_en_new = np.stack(df_pairs_evaluate["padding_en"].values)
features_jp_new = np.stack(df_pairs_evaluate["padding_jp"].values)
features_en_new_wrong = np.stack(df_pairs_evaluate["padding_en_wrong"].values)

Using the new test data to evaluate.......


/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [237]:
features_en_new.shape

(1000, 8, 100)

In [238]:
df_pairs_sample

,en_article,jp_article,similarity,dis_similarity,en_article_wrong,word2vec_en,word2vec_en_wrong,word2vec_jp,padding_en,padding_en_wrong,padding_jp
0,china slow mid-term prospect bright-fitch,中国 経済成長 やや 減速 中期 見通し 良好 フィッチ,5.0,1.0,factbox-electoral vote tally US election,"[[-0.390865, 0.397, -1.29951, -0.392807, 0.076...","[[-0.116757, -0.127333, 0.204791, -0.152131, 0...","[[0.402830287324, 0.129367016595, -0.107190614...","[[-0.390864849091, 0.397000223398, -1.29950547...","[[-0.116756603122, -0.127333447337, 0.20479136...","[[0.402830287324, 0.129367016595, -0.107190614..."
1,US stock open low ahead greenspan remark,米国 株式市場 序盤 小幅安 グリーンスパン 議長 証言 控え,5.0,1.0,hk share slip open financials weigh,"[[0.0994734, 0.454259, -0.405963, -0.548646, -...","[[-0.405822, 0.588406, -0.629584, -0.55204, 0....","[[0.348488901316, 0.196023231599, 0.0017346462...","[[0.0994734168053, 0.454259365797, -0.40596297...","[[-0.40582165122, 0.588405668736, -0.629583835...","[[0.348488901316, 0.196023231599, 0.0017346462..."
2,US 3-month bill high rate pct,表 米 落札 結果 最高 落札 金利 3か月 6か月,5.0,1.0,analysis-long bumpy road ahead obama financial...,"[[-0.480557, 0.133917, -0.418268, -0.314347, -...","[[-0.109783, -0.0746828, -0.450799, -0.251951,...","[[0.616814012804, 0.16483522273, -0.1424001951...","[[-0.480556935072, 0.133917048573, -0.41826826...","[[-0.109783045948, -0.0746828317642, -0.450799...","[[0.616814012804, 0.16483522273, -0.1424001951..."
3,US 3-month bill high rate pct,表 米 落札 結果 最高 落札 金利 3か月 6か月,5.0,1.0,us treasury outlook-headline risk,"[[-0.480557, 0.133917, -0.418268, -0.314347, -...","[[-0.327858, 0.188393, -0.14006, 0.16072, -0.3...","[[0.616814012804, 0.16483522273, -0.1424001951...","[[-0.480556935072, 0.133917048573, -0.41826826...","[[-0.32785820961, 0.188392534852, -0.140060022...","[[0.616814012804, 0.16483522273, -0.1424001951..."
4,US 3-month bill high rate pct,表 米 落札 結果 最高 落札 金利 3か月 6か月,5.0,1.0,instant view 7- fed refrain cut stimulus,"[[-0.480557, 0.133917, -0.418268, -0.314347, -...","[[-1.23364, 0.219623, -0.724582, -0.373339, -1...","[[0.616814012804, 0.16483522273, -0.1424001951...","[[-0.480556935072, 0.133917048573, -0.41826826...","[[-1.23364293575, 0.219622790813, -0.724581539...","[[0.616814012804, 0.16483522273, -0.1424001951..."
5,US stock open slightly high ahead fed,米国 株式市場 序盤 小 反発 FOMC 結果 見守る,5.0,1.0,highlights-bernanke testimony fed monetary p...,"[[0.0994734, 0.454259, -0.405963, -0.548646, -...","[[-0.145878, -0.230469, -0.198219, 0.224067, -...","[[0.348488901316, 0.196023231599, 0.0017346462...","[[0.0994734168053, 0.454259365797, -0.40596297...","[[-0.145877897739, -0.230468779802, -0.1982191...","[[0.348488901316, 0.196023231599, 0.0017346462..."
6,rpt-US stock open slightly high ahead fed,米国 株式市場 序盤 小 反発 FOMC 結果 見守る,5.0,1.0,coeure say ecb keep monetary policy accommodative,"[[0.0994734, 0.454259, -0.405963, -0.548646, -...","[[0.157944, 0.111231, -0.0946961, -0.450471, -...","[[0.348488901316, 0.196023231599, 0.0017346462...","[[0.0994734168053, 0.454259365797, -0.40596297...","[[0.157944068313, 0.111230537295, -0.094696149...","[[0.348488901316, 0.196023231599, 0.0017346462..."
7,blair say worry oil want low price,原油価格 問題 議題 なる 可能性 英 首相,5.0,1.0,wrapup 2-china order state firm explore potash...,"[[0.100457, 0.192785, -0.073684, -0.460842, 0....","[[-0.723512, -0.139943, -0.52488, -0.0200835, ...","[[0.535308593675, -0.0156275289532, 0.29363014...","[[0.100456625223, 0.192785292864, -0.073683954...","[[-0.723511993885, -0.139943093061, -0.5248797...","[[0.535308593675, -0.0156275289532, 0.29363014..."
8,s korea fin min see gdp 4-5 pct report,今年 下期 韓国 GDP 成長率 見通し 財政 経済,5.0,1.0,bmw post may sale record volumes up pct,"[[-0.491196, 0.482589, -0.213298, -0.0388717, ...","[[-0.242481, 0.692613, -0.13031, -0.738784, -0...","[[0.418770197027, 0.0354079812488, 0.065243760...","[[-0.491196334362, 0.482588618994, -0.21329811...","[[-0.242481142282, 0.692612826824, -0.13031049...","[[0.418770197027, 0.0354079812488, 0.0652

In [239]:
# ---- Training data ---- #
df_pairs_sample = df_pairs_enjp.iloc[0:50000]

df_pairs_sample['word2vec_en'] = df_pairs_sample['en_article'].apply(doc2embed,args=(model_en,))
df_pairs_sample['word2vec_en_wrong'] = df_pairs_sample['en_article_wrong'].apply(doc2embed,args=(model_en,))
df_pairs_sample['word2vec_jp'] = df_pairs_sample['jp_article'].apply(doc2embed,args=(model_jp,trans_jp_en))


# ---- Padding the vector ---- #
df_pairs_sample['padding_en'] = df_pairs_sample['word2vec_en'].apply(padding, args=(TIME_STEP,))
df_pairs_sample['padding_en_wrong'] = df_pairs_sample['word2vec_en_wrong'].apply(padding, args=(TIME_STEP,))
df_pairs_sample['padding_jp'] = df_pairs_sample['word2vec_jp'].apply(padding, args=(TIME_STEP,))

df_pairs_sample.dropna(axis=0, how='all')



/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

长度为零
长度为零
长度为零
长度为零
长度为零
长度为零
长度为零
长度为零
长度为零
长度为零
长度为零
长度为零
长度为零
长度为零


/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


长度为零
长度为零
长度为零
长度为零
长度为零
长度为零
长度为零
长度为零
长度为零


/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,en_article,jp_article,similarity,dis_similarity,en_article_wrong,word2vec_en,word2vec_en_wrong,word2vec_jp,padding_en,padding_en_wrong,padding_jp
0,china slow mid-term prospect bright-fitch,中国 経済成長 やや 減速 中期 見通し 良好 フィッチ,5.0,1.0,scenarios-hot topic market finance meeting,"[[-0.390865, 0.397, -1.29951, -0.392807, 0.076...","[[0.675374, -0.000147757, 0.153712, -0.80408, ...","[[0.402830287324, 0.129367016595, -0.107190614...","[[-0.390864849091, 0.397000223398, -1.29950547...","[[0.675373613834, -0.000147756596562, 0.153712...","[[0.402830287324, 0.129367016595, -0.107190614..."
1,US stock open low ahead greenspan remark,米国 株式市場 序盤 小幅安 グリーンスパン 議長 証言 控え,5.0,1.0,instant view 6-job growth slow expect jan,"[[0.0994734, 0.454259, -0.405963, -0.548646, -...","[[-1.23364, 0.219623, -0.724582, -0.373339, -1...","[[0.348488901316, 0.196023231599, 0.0017346462...","[[0.0994734168053, 0.454259365797, -0.40596297...","[[-1.23364293575, 0.219622790813, -0.724581539...","[[0.348488901316, 0.196023231599, 0.0017346462..."
2,US 3-month bill high rate pct,表 米 落札 結果 最高 落札 金利 3か月 6か月,5.0,1.0,poll-us aug retail sale seEUp pct,"[[-0.480557, 0.133917, -0.418268, -0.314347, -...","[[-0.337856, 0.788899, -0.0348818, 0.0216262, ...","[[0.616814012804, 0.16483522273, -0.1424001951...","[[-0.480556935072, 0.133917048573, -0.41826826...","[[-0.337856441736, 0.788898706436, -0.03488179...","[[0.616814012804, 0.16483522273, -0.1424001951..."
3,US 3-month bill high rate pct,表 米 落札 結果 最高 落札 金利 3か月 6か月,5.0,1.0,bank canada cut rate cite downside risk,"[[-0.480557, 0.133917, -0.418268, -0.314347, -...","[[0.259398, 0.730876, 0.0491157, -1.17238, -0....","[[0.616814012804, 0.16483522273, -0.1424001951...","[[-0.480556935072, 0.133917048573, -0.41826826...","[[0.259397864342, 0.730876386166, 0.0491157025...","[[0.616814012804, 0.16483522273, -0.1424001951..."
4,US 3-month bill high rate pct,表 米 落札 結果 最高 落札 金利 3か月 6か月,5.0,1.0,stock snapshot-wall st jump ge rating outlook ...,"[[-0.480557, 0.133917, -0.418268, -0.314347, -...","[[0.0994734, 0.454259, -0.405963, -0.548646, -...","[[0.616814012804, 0.16483522273, -0.1424001951...","[[-0.480556935072, 0.133917048573, -0.41826826...","[[0.0994734168053, 0.454259365797, -0.40596297...","[[0.616814012804, 0.16483522273, -0.1424001951..."
5,US stock open slightly high ahead fed,米国 株式市場 序盤 小 反発 FOMC 結果 見守る,5.0,1.0,gm say significant interest asset sale,"[[0.0994734, 0.454259, -0.405963, -0.548646, -...","[[-0.371791, 0.545951, -0.136044, -0.804635, -...","[[0.348488901316, 0.196023231599, 0.0017346462...","[[0.0994734168053, 0.454259365797, -0.40596297...","[[-0.371791392565, 0.545950770378, -0.13604432...","[[0.348488901316, 0.196023231599, 0.0017346462..."
6,rpt-US stock open slightly high ahead fed,米国 株式市場 序盤 小 反発 FOMC 結果 見守る,5.0,1.0,imf say split eu ecb greece debt,"[[0.0994734, 0.454259, -0.405963, -0.548646, -...","[[0.144858, -0.594477, -0.208458, -0.954657, -...","[[0.348488901316, 0.196023231599, 0.0017346462...","[[0.0994734168053, 0.454259365797, -0.40596297...","[[0.144857540727, -0.594477176666, -0.20845755...","[[0.348488901316, 0.196023231599, 0.0017346462..."
7,blair say worry oil want low price,原油価格 問題 議題 なる 可能性 英 首相,5.0,1.0,factbox-electoral vote tally US election,"[[0.100457, 0.192785, -0.073684, -0.460842, 0....","[[-0.116757, -0.127333, 0.204791, -0.152131, 0...","[[0.535308593675, -0.0156275289532, 0.29363014...","[[0.100456625223, 0.192785292864, -0.073683954...","[[-0.116756603122, -0.127333447337, 0.20479136...","[[0.535308593675, -0.0156275289532, 0.29363014..."
8,s korea fin min see gdp 4-5 pct report,今年 下期 韓国 GDP 成長率 見通し 財政 経済,5.0,1.0,poll-boe cut again feb rate trough,"[[-0.491196, 0.482589, -0.213298, -0.0388717, ...","[[-0.374289, 0.129752, -0.255022, -0.514244, 0...","[[0.418770197027, 0.0354079812488, 0.065243760...","[[-0.491196, 0.482589, -0.213298, -0.0388717, ...","[[-0.374288797379, 0.129751950502, -0.25502192...","[[0.418770197027, 0.0354079812488, 0.065243760..."
9,s korea fin min see g

In [240]:
df_pairs_sample = df_pairs_sample[df_pairs_sample['word2vec_en']!=float('nan')]

In [241]:
df_pairs_sample

,en_article,jp_article,similarity,dis_similarity,en_article_wrong,word2vec_en,word2vec_en_wrong,word2vec_jp,padding_en,padding_en_wrong,padding_jp
0,china slow mid-term prospect bright-fitch,中国 経済成長 やや 減速 中期 見通し 良好 フィッチ,5.0,1.0,scenarios-hot topic market finance meeting,"[[-0.390865, 0.397, -1.29951, -0.392807, 0.076...","[[0.675374, -0.000147757, 0.153712, -0.80408, ...","[[0.402830287324, 0.129367016595, -0.107190614...","[[-0.390864849091, 0.397000223398, -1.29950547...","[[0.675373613834, -0.000147756596562, 0.153712...","[[0.402830287324, 0.129367016595, -0.107190614..."
1,US stock open low ahead greenspan remark,米国 株式市場 序盤 小幅安 グリーンスパン 議長 証言 控え,5.0,1.0,instant view 6-job growth slow expect jan,"[[0.0994734, 0.454259, -0.405963, -0.548646, -...","[[-1.23364, 0.219623, -0.724582, -0.373339, -1...","[[0.348488901316, 0.196023231599, 0.0017346462...","[[0.0994734168053, 0.454259365797, -0.40596297...","[[-1.23364293575, 0.219622790813, -0.724581539...","[[0.348488901316, 0.196023231599, 0.0017346462..."
2,US 3-month bill high rate pct,表 米 落札 結果 最高 落札 金利 3か月 6か月,5.0,1.0,poll-us aug retail sale seEUp pct,"[[-0.480557, 0.133917, -0.418268, -0.314347, -...","[[-0.337856, 0.788899, -0.0348818, 0.0216262, ...","[[0.616814012804, 0.16483522273, -0.1424001951...","[[-0.480556935072, 0.133917048573, -0.41826826...","[[-0.337856441736, 0.788898706436, -0.03488179...","[[0.616814012804, 0.16483522273, -0.1424001951..."
3,US 3-month bill high rate pct,表 米 落札 結果 最高 落札 金利 3か月 6か月,5.0,1.0,bank canada cut rate cite downside risk,"[[-0.480557, 0.133917, -0.418268, -0.314347, -...","[[0.259398, 0.730876, 0.0491157, -1.17238, -0....","[[0.616814012804, 0.16483522273, -0.1424001951...","[[-0.480556935072, 0.133917048573, -0.41826826...","[[0.259397864342, 0.730876386166, 0.0491157025...","[[0.616814012804, 0.16483522273, -0.1424001951..."
4,US 3-month bill high rate pct,表 米 落札 結果 最高 落札 金利 3か月 6か月,5.0,1.0,stock snapshot-wall st jump ge rating outlook ...,"[[-0.480557, 0.133917, -0.418268, -0.314347, -...","[[0.0994734, 0.454259, -0.405963, -0.548646, -...","[[0.616814012804, 0.16483522273, -0.1424001951...","[[-0.480556935072, 0.133917048573, -0.41826826...","[[0.0994734168053, 0.454259365797, -0.40596297...","[[0.616814012804, 0.16483522273, -0.1424001951..."
5,US stock open slightly high ahead fed,米国 株式市場 序盤 小 反発 FOMC 結果 見守る,5.0,1.0,gm say significant interest asset sale,"[[0.0994734, 0.454259, -0.405963, -0.548646, -...","[[-0.371791, 0.545951, -0.136044, -0.804635, -...","[[0.348488901316, 0.196023231599, 0.0017346462...","[[0.0994734168053, 0.454259365797, -0.40596297...","[[-0.371791392565, 0.545950770378, -0.13604432...","[[0.348488901316, 0.196023231599, 0.0017346462..."
6,rpt-US stock open slightly high ahead fed,米国 株式市場 序盤 小 反発 FOMC 結果 見守る,5.0,1.0,imf say split eu ecb greece debt,"[[0.0994734, 0.454259, -0.405963, -0.548646, -...","[[0.144858, -0.594477, -0.208458, -0.954657, -...","[[0.348488901316, 0.196023231599, 0.0017346462...","[[0.0994734168053, 0.454259365797, -0.40596297...","[[0.144857540727, -0.594477176666, -0.20845755...","[[0.348488901316, 0.196023231599, 0.0017346462..."
7,blair say worry oil want low price,原油価格 問題 議題 なる 可能性 英 首相,5.0,1.0,factbox-electoral vote tally US election,"[[0.100457, 0.192785, -0.073684, -0.460842, 0....","[[-0.116757, -0.127333, 0.204791, -0.152131, 0...","[[0.535308593675, -0.0156275289532, 0.29363014...","[[0.100456625223, 0.192785292864, -0.073683954...","[[-0.116756603122, -0.127333447337, 0.20479136...","[[0.535308593675, -0.0156275289532, 0.29363014..."
8,s korea fin min see gdp 4-5 pct report,今年 下期 韓国 GDP 成長率 見通し 財政 経済,5.0,1.0,poll-boe cut again feb rate trough,"[[-0.491196, 0.482589, -0.213298, -0.0388717, ...","[[-0.374289, 0.129752, -0.255022, -0.514244, 0...","[[0.418770197027, 0.0354079812488, 0.065243760...","[[-0.491196, 0.482589, -0.213298, -0.0388717, ...","[[-0.374288797379, 0.129751950502, -0.25502192...","[[0.418770197027, 0.0354079812488, 0.065243760..."
9,s korea fin min see g

In [242]:
# Generate training data (similarity = 1)
features_en_1 = np.array(df_pairs_sample["padding_en"].values.tolist(),dtype=np.float32)
features_jp_1 = np.array(df_pairs_sample["padding_jp"].values.tolist(),dtype=np.float32)

# Generate training data (similarity = 0)
features_en_0 = np.array(df_pairs_sample["padding_en_wrong"].values.tolist(),dtype=np.float32)
#np.random.shuffle((features_en_0))

In [243]:
features_jp_1.shape

(50000, 8, 100)

In [261]:
torch.manual_seed(1)    # reproducible

# Hyper Parameters
EPOCH = 1          # 训练整批数据多少次
BATCH_SIZE = 100
TIME_STEP = 20      # rnn 时间步数 / 图片高度
INPUT_SIZE = 100     # rnn 每步输入值 / 图片每行像素
HIDDEN_SIZE = 100
BIDIRECTION = False
LR = 1e-4           # learning rate
DROP_OUT = 0.0

In [ ]:
class LaLSTM(nn.Module):
    def __init__(self, batch_size=BATCH_SIZE, time_step=TIME_STEP, hidden_size=HIDDEN_SIZE, input_size=INPUT_SIZE, dropout_p=DROP_OUT, bidirection=False, attention=False):
        super(LaLSTM, self).__init__()
        self.attention = attention
        self.lstm_left = nn.LSTM(     # LSTM 效果要比 nn.RNN() 好多了
            input_size=input_size,      # 图片每行的数据像素点
            hidden_size=hidden_size,     # rnn hidden unit
            num_layers=1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
            bidirectional=bidirection,
        )
        
        self.lstm_right = nn.LSTM(     # LSTM 效果要比 nn.RNN() 好多了
            input_size=input_size,      # 图片每行的数据像素点
            hidden_size=hidden_size,     # rnn hidden unit
            num_layers=1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
            bidirectional=bidirection,
        )
        
        if bidirection:
            print("bidirection")
            self.full_connect = torch.nn.Sequential(
                                                #torch.nn.Linear(hidden_size*2, hidden_size),
                                                #torch.nn.ReLU(),
                                                torch.nn.Linear(hidden_size*4, 1),
                                                torch.nn.ReLU()
                                                )
            self.attn_left = nn.Linear(4*hidden_size, hidden_size, bias=False)
            #self.attn_apply_left = nn.Linear(self.)
            self.attn_right = nn.Linear(4*hidden_size, hidden_size, bias=False)
        else:
            self.attn_left = nn.Linear(2*hidden_size, hidden_size, bias=False)
            #self.attn_apply_left = nn.Linear(self.)
            self.attn_right = nn.Linear(2*hidden_size, hidden_size, bias=False)

            self.full_connect = torch.nn.Sequential(
                                                    #torch.nn.Linear(hidden_size*2, hidden_size),
                                                    #torch.nn.ReLU(),
                                                    torch.nn.Linear(hidden_size*2, 1),
                                                    torch.nn.Sigmoid()
                                                    )

        self.dropout = nn.Dropout(dropout_p)
        #self.gru = nn.GRU(self.hidden_size, self.hidden_size)

        nn.init.xavier_uniform(self.conv1.weight)
    def forward(self, x_left, x_right):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)   LSTM 有两个 hidden states, h_n 是分线, h_c 是主线
        # h_c shape (n_layers, batch, hidden_size)
        # batch_first = True : (batch, time_step, input_size)
        
        ##left_out, (h_n_left, h_c_left) = self.lstm_left(x, None)   # None 表示 hidden state 会用全0的 state
        ##right_out, (h_n_right, h_c_right) = self.lstm_right(x, None)

        # 选取最后一个时间点的 r_out 输出
        # 这里 r_out[:, -1, :] 的值也是 h_n 的值
        ##out = self.out(r_out[:, -1, :])
        ##return out
    
        left_out, left_hidden = self.lstm_left(x_left)   # None 表示 hidden state 会用全0的 state
        right_out, right_hidden = self.lstm_right(x_right)
        #r_out_reshaped = r_out.view(-1, HIDDEN_SIZE) # to 2D data
        #outs = self.linear_layer(r_out_reshaped)
        #left_out = left_out.view(-1, TIME_STEP, INPUT_SIZE)  # to 3D data
        #right_out = right_out.view(-1, TIME_STEP, INPUT_SIZE)  # to 3D data
        
        #h_state_left = (h_n_left, h_c_left)
        #h_state_right = (h_n_right, h_c_right)
        
        # attention = h_left dot h_right.T
        #(batch, h_left, h_right)
        attn_matrix_left = torch.bmm(left_out,torch.transpose(right_out,1,2))
        #(batch, h_right, h_left)
        attn_matrix_right = torch.bmm(right_out,torch.transpose(left_out,1,2))
        left_at = F.softmax(attn_matrix_left,2)
        right_at = F.softmax(attn_matrix_right,2)
        left_ct = torch.bmm(left_at,right_out)
        right_ct = torch.bmm(right_at,left_out)
        #take only last attention for results without feed_in
        left_hn = left_out[:, -1, :].squeeze(1)
        right_hn = right_out[:, -1, :].squeeze(1)
        left_cn = left_ct[:, -1, :].squeeze(1)
        right_cn = right_ct[:, -1, :].squeeze(1)
        
        left_new_hn = F.tanh(self.attn_left(torch.cat((left_hn,left_cn),1)))
        right_new_hn = F.tanh(self.attn_right(torch.cat((right_hn,right_cn),1)))
        
        if self.attention:
            self.out = self.full_connect(torch.cat((left_new_hn,right_new_hn),1))
        else:
            self.out = self.full_connect(torch.cat((left_hn,right_hn),1))
        
        return self.out

lalstm = LaLSTM()
print(lalstm)

In [ ]:
model = LaLSTM(bidirection=True,attention=False)
train_feature_left = np.concatenate((features_en_1,features_en_0),axis=0)
train_feature_right = np.concatenate((features_jp_1,features_jp_1),axis=0)
train_result = np.concatenate((np.ones(features_en_1.shape[0],dtype=np.float32),np.zeros(features_en_0.shape[0],dtype=np.float32)),axis=0)
train_result = train_result[:,np.newaxis]

p = np.random.permutation(len(train_result))
train_feature_left = train_feature_left[p]
train_feature_right = train_feature_right[p]
train_result = train_result[p]

print(train_feature_left.shape,train_feature_right.shape,train_result.shape)

# different optimizers
opt_SGD         = torch.optim.SGD(model.parameters(), lr=LR)
opt_Momentum    = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.8)
opt_RMSprop     = torch.optim.RMSprop(model.parameters(), lr=LR, alpha=0.9)
opt_Adam        = torch.optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.99))

criterion = torch.nn.MSELoss(size_average=False)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
optimizer = opt_Adam

all_loss = []
for epoch in range(EPOCH):
    loop = train_result.shape[0]//BATCH_SIZE
    for t in range(loop):
        # Forward pass: Compute predicted y by passing x to the model
        in_left = torch.from_numpy(train_feature_left[BATCH_SIZE*t:BATCH_SIZE*(t+1)])
        in_right = torch.from_numpy(train_feature_right[BATCH_SIZE*t:BATCH_SIZE*(t+1)])
        output = torch.from_numpy(train_result[BATCH_SIZE*t:BATCH_SIZE*(t+1)])
        y_pred = model(in_left,in_right)

        # Compute and print loss
        loss = criterion(y_pred, output)
        all_loss.append(loss.item())
        print(t, loss.item())

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
plt.plot(all_loss)

bidirection
(100000, 8, 100) (100000, 8, 100) (100000, 1)
0 53.869354248046875
1 42.00189208984375
2 51.850074768066406
3 43.84453201293945
4 55.52882766723633
5 59.58960723876953
6 51.28359603881836
7 62.81837463378906
8 46.5263671875
9 39.27464294433594
10 44.368770599365234
11 49.632423400878906
12 54.241268157958984
13 37.43450164794922
14 47.43622970581055
15 35.58577346801758
16 45.09612274169922
17 44.48846435546875
18 42.49979782104492
19 39.57989501953125
20 35.933570861816406
21 42.52427673339844
22 38.74758529663086
23 42.66655731201172
24 40.69540023803711
25 40.986995697021484
26 43.81574630737305
27 34.073036193847656
28 38.983642578125
29 36.34511947631836
30 30.27325439453125
31 34.6392707824707
32 41.77785110473633
33 35.656463623046875
34 34.3630485534668
35 36.2239990234375
36 34.34028244018555
37 27.37677764892578
38 29.752239227294922
39 30.635526657104492
40 33.091941833496094
41 30.619457244873047
42 33.45885467529297
43 24.673015594482422
44 31.1241455078125
45 

In [256]:
model_att = LaLSTM(bidirection=False,attention=True)
print(train_feature_left.shape,train_feature_right.shape,train_result.shape)

criterion = torch.nn.MSELoss(size_average=False)
#optimizer = torch.optim.SGD(model_att.parameters(), lr=LR, momentum=0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.99))

all_loss = []
for epoch in range(EPOCH):
    loop = train_result.shape[0]//BATCH_SIZE
    for t in range(loop):
        # Forward pass: Compute predicted y by passing x to the model
        in_left = torch.from_numpy(train_feature_left[BATCH_SIZE*t:BATCH_SIZE*(t+1)])
        in_right = torch.from_numpy(train_feature_right[BATCH_SIZE*t:BATCH_SIZE*(t+1)])
        output = torch.from_numpy(train_result[BATCH_SIZE*t:BATCH_SIZE*(t+1)])
        y_pred = model_att(in_left,in_right)

        # Compute and print loss
        loss = criterion(y_pred, output)
        all_loss.append(loss.item())
        print(t, loss.item())

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
plt.plot(all_loss)

(100000, 8, 100) (100000, 8, 100) (100000, 1)
0 25.009380340576172
1 25.038726806640625
2 24.9663143157959
3 25.001609802246094
4 24.974300384521484
5 24.99850845336914
6 25.003162384033203
7 24.977928161621094
8 25.01534652709961
9 25.016300201416016
10 25.011260986328125
11 24.945369720458984
12 24.975444793701172
13 25.00014305114746
14 24.9792423248291
15 25.01626205444336
16 25.020139694213867
17 25.010173797607422
18 24.962039947509766
19 24.99256706237793
20 25.036893844604492
21 24.98285675048828
22 24.98672866821289
23 24.989086151123047
24 24.98230743408203
25 25.015586853027344
26 24.981313705444336
27 24.95572280883789
28 25.03152084350586
29 24.94442367553711
30 24.999658584594727
31 24.97598648071289
32 24.962743759155273
33 25.013225555419922
34 25.041629791259766
35 24.90897560119629
36 25.039003372192383
37 24.98619270324707
38 24.98108673095703
39 24.976436614990234
40 24.99800682067871
41 24.943750381469727
42 25.015470504760742
43 24.97239112854004
44 24.97180557250

370 24.985475540161133
371 25.00143051147461
372 24.943035125732422
373 24.96016502380371
374 24.999780654907227
375 25.0128173828125
376 24.989408493041992
377 25.021812438964844
378 24.99195098876953
379 25.001516342163086
380 24.943010330200195
381 25.00418472290039
382 24.985315322875977
383 24.98690414428711
384 24.971214294433594
385 25.00058937072754
386 24.96218490600586
387 24.94573974609375
388 25.037567138671875
389 25.047630310058594
390 24.967416763305664
391 24.983015060424805
392 24.975664138793945
393 25.004390716552734
394 24.983413696289062
395 24.98385238647461
396 25.00177001953125
397 24.993305206298828
398 25.03658676147461
399 25.018205642700195
400 25.018943786621094
401 24.96112632751465
402 24.96894645690918
403 25.02914047241211
404 25.000118255615234
405 24.972179412841797
406 24.983644485473633
407 25.01260757446289
408 24.970661163330078
409 25.012561798095703
410 25.044212341308594
411 25.066011428833008
412 25.02962303161621
413 25.01652717590332
414 25.

732 25.00686264038086
733 25.0035457611084
734 25.066282272338867
735 24.903207778930664
736 24.97877311706543
737 24.983901977539062
738 24.98221206665039
739 24.93539810180664
740 24.996789932250977
741 25.001705169677734
742 24.977476119995117
743 25.04178810119629
744 24.963211059570312
745 25.015806198120117
746 25.025623321533203
747 24.99227523803711
748 24.948196411132812
749 25.024133682250977
750 25.034400939941406
751 25.0361270904541
752 24.984397888183594
753 25.00733184814453
754 24.99153709411621
755 25.001304626464844
756 24.988643646240234
757 24.99563980102539
758 24.98017692565918
759 24.955204010009766
760 24.98029327392578
761 24.979995727539062
762 24.985666275024414
763 25.056411743164062
764 25.000900268554688
765 24.99455451965332
766 25.047775268554688
767 25.04595184326172
768 24.980140686035156
769 25.00535774230957
770 25.05942153930664
771 25.042078018188477
772 24.99524688720703
773 25.027008056640625
774 24.97435760498047
775 25.023998260498047
776 24.98

104 24.96739959716797
105 25.022825241088867
106 25.001943588256836
107 24.995357513427734
108 24.954059600830078
109 24.956396102905273
110 25.024328231811523
111 24.936017990112305
112 25.023454666137695
113 24.98587417602539
114 24.9525203704834
115 24.980857849121094
116 25.013097763061523
117 24.951601028442383
118 24.997194290161133
119 25.016115188598633
120 24.95039176940918
121 24.975025177001953
122 25.00382423400879
123 25.018054962158203
124 25.043033599853516
125 25.018146514892578
126 24.988262176513672
127 24.96346664428711
128 24.978818893432617
129 25.08391571044922
130 25.023155212402344
131 24.98980140686035
132 24.955472946166992
133 24.9560489654541
134 25.002321243286133


KeyboardInterrupt: 

In [ ]:
model_plain = LaLSTM(bidirection=False,attention=False)
print(train_feature_left.shape,train_feature_right.shape,train_result.shape)

criterion = torch.nn.MSELoss(size_average=False)
#optimizer = torch.optim.SGD(model_att.parameters(), lr=LR, momentum=0.2)
optimizer = torch.optim.Adam(model_plain.parameters(), lr=LR, betas=(0.9, 0.99))

all_loss = []
for epoch in range(EPOCH):
    loop = train_result.shape[0]//BATCH_SIZE
    for t in range(loop):
        # Forward pass: Compute predicted y by passing x to the model
        in_left = torch.from_numpy(train_feature_left[BATCH_SIZE*t:BATCH_SIZE*(t+1)])
        in_right = torch.from_numpy(train_feature_right[BATCH_SIZE*t:BATCH_SIZE*(t+1)])
        output = torch.from_numpy(train_result[BATCH_SIZE*t:BATCH_SIZE*(t+1)])
        
        #in_left = in_left.to(torch.device("cuda"))
        #in_right = in_rightt.to(torch.device("cuda"))
        #output = output.to(torch.device("cuda"))
        
        
        y_pred = model_plain(in_left,in_right)
        # Compute and print loss
        loss = criterion(y_pred, output)
        all_loss.append(loss.item())
        print(t, loss.item())

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
plt.plot(all_loss)

(100000, 8, 100) (100000, 8, 100) (100000, 1)
0 25.10033416748047
1 25.031898498535156
2 24.82028579711914
3 24.890762329101562
4 25.039777755737305
5 25.104433059692383
6 24.99911117553711
7 24.87327766418457
8 25.009950637817383
9 24.78809356689453
10 25.068607330322266
11 24.833518981933594
12 25.196134567260742
13 25.579803466796875
14 25.133766174316406
15 25.109346389770508
16 24.925533294677734
17 25.134197235107422
18 25.289072036743164
19 24.836822509765625
20 24.861614227294922
21 25.07648468017578
22 25.074893951416016
23 25.005443572998047
24 25.027381896972656
25 25.01430892944336
26 24.847301483154297
27 25.149993896484375
28 24.803478240966797
29 24.877971649169922
30 25.037506103515625
31 25.159454345703125
32 24.869701385498047
33 25.084745407104492
34 25.026639938354492
35 24.921812057495117
36 24.914104461669922
37 25.07965087890625
38 24.813283920288086
39 24.9095401763916
40 25.159114837646484
41 24.95729637145996
42 25.01707649230957
43 25.023160934448242
44 24.78

366 23.693456649780273
367 25.399261474609375
368 25.143030166625977
369 24.316146850585938
370 24.9623966217041
371 24.998815536499023
372 24.299816131591797
373 24.031457901000977
374 25.127878189086914
375 25.205230712890625
376 24.316936492919922
377 25.445037841796875
378 25.276718139648438
379 24.314144134521484
380 24.937023162841797
381 24.840469360351562
382 25.29792594909668
383 24.194332122802734
384 24.788373947143555
385 24.250173568725586
386 25.217863082885742
387 24.14793586730957
388 25.268016815185547
389 25.640581130981445
390 24.819660186767578
391 24.70261573791504
392 24.70403289794922
393 24.791168212890625
394 24.820005416870117
395 25.145029067993164
396 24.633769989013672
397 24.32481575012207
398 24.8776798248291
399 23.98041343688965
400 24.914112091064453
401 24.980026245117188
402 24.378660202026367
403 24.90724754333496
404 24.140804290771484
405 24.328508377075195
406 24.655948638916016
407 24.2261962890625
408 24.62814712524414
409 24.996023178100586
41

730 23.847848892211914
731 24.311145782470703
732 24.481138229370117
733 24.346248626708984
734 24.837955474853516
735 24.833660125732422
736 25.644710540771484
737 24.690189361572266
738 24.54368019104004
739 24.329917907714844
740 24.68972396850586
741 24.95005226135254
742 24.742107391357422
743 24.134010314941406
744 23.637802124023438
745 24.41325569152832
746 24.3343563079834
747 25.21226692199707
748 24.154010772705078
749 24.389827728271484
750 25.445646286010742
751 24.656492233276367
752 24.963956832885742
753 24.6301326751709
754 24.36811637878418
755 23.33008575439453
756 24.678932189941406
757 25.20830726623535
758 23.74123191833496
759 24.963926315307617
760 24.65288734436035
761 23.685272216796875
762 24.931032180786133
763 24.836538314819336
764 25.678665161132812
765 24.550466537475586
766 24.467174530029297
767 24.268875122070312
768 25.35675621032715
769 24.043394088745117
770 24.493749618530273
771 25.597938537597656
772 23.880144119262695
773 24.528202056884766
774

97 23.574783325195312
98 24.71465301513672
99 24.108949661254883
100 23.216510772705078
101 25.042274475097656
102 23.78908920288086
103 24.599002838134766
104 24.70871925354004
105 23.41468620300293
106 24.147005081176758
107 24.72364044189453
108 25.42859649658203
109 25.278223037719727
110 23.34905242919922
111 24.174837112426758
112 24.056068420410156
113 23.655729293823242
114 25.320796966552734
115 23.581356048583984
116 25.321014404296875
117 24.498565673828125
118 23.67653465270996
119 25.130699157714844
120 24.073986053466797
121 25.054603576660156
122 24.899173736572266
123 25.29482650756836
124 24.52080535888672
125 22.007707595825195
126 22.55816078186035
127 24.382192611694336
128 24.54648208618164
129 24.972518920898438
130 24.153690338134766
131 23.776403427124023
132 23.921680450439453
133 23.92130470275879
134 24.337831497192383
135 25.293460845947266
136 24.207860946655273
137 24.217987060546875
138 25.252498626708984
139 24.19571876525879
140 25.040287017822266
141 2

461 24.499757766723633
462 24.976295471191406
463 23.981733322143555
464 24.516876220703125
465 23.1948184967041
466 24.262842178344727
467 24.165626525878906
468 23.526939392089844
469 23.202510833740234
470 24.265575408935547
471 24.220367431640625
472 22.8172607421875
473 24.076129913330078
474 24.367578506469727
475 23.667451858520508
476 23.09994125366211
477 25.621845245361328
478 24.750022888183594
479 24.24292755126953
480 24.10550880432129
481 24.7382755279541
482 23.559406280517578
483 23.739545822143555
484 25.6190242767334
485 24.70538902282715
486 24.543834686279297
487 23.75801658630371
488 23.425750732421875
489 25.023038864135742
490 23.717124938964844
491 23.241243362426758
492 24.295372009277344
493 25.698606491088867
494 22.580265045166016
495 24.991323471069336
496 24.998563766479492
497 26.765357971191406
498 23.93451690673828
499 24.561037063598633
500 24.63911247253418
501 24.36691665649414
502 25.580001831054688
503 24.253952026367188
504 22.846168518066406
505 

...

390 24.683332443237305
391 23.789926528930664
392 22.711519241333008
393 23.659595489501953
394 24.33852767944336
395 24.772817611694336
396 23.80571746826172
397 24.96120834350586
398 24.15839385986328
399 22.758525848388672
400 23.42410659790039
401 25.011131286621094
402 23.119338989257812
403 23.75298309326172
404 23.29227638244629
405 21.765827178955078
406 23.688297271728516
407 23.885448455810547
408 25.837186813354492
409 22.9681453704834
410 23.82554817199707
411 26.186233520507812
412 23.269838333129883
413 25.17233657836914
414 25.508455276489258
415 25.977243423461914
416 23.676788330078125
417 24.565927505493164
418 23.494617462158203
419 23.306049346923828
420 23.081560134887695
421 23.95020294189453
422 23.802593231201172
423 24.662738800048828
424 24.587352752685547
425 23.17415428161621
426 23.918113708496094
427 23.41162109375
428 23.71287727355957
429 22.90503692626953
430 23.07525634765625
431 25.989328384399414
432 23.52665138244629
433 23.997554779052734
434 23.19

755 21.850852966308594
756 24.317716598510742
757 25.304533004760742
758 21.91428565979004
759 23.939189910888672
760 23.852567672729492
761 23.13804054260254
762 25.801788330078125
763 22.533281326293945
764 24.679853439331055
765 23.695579528808594
766 24.278627395629883
767 24.28520965576172
768 25.729555130004883
769 22.984363555908203
770 24.646629333496094
771 25.863231658935547
772 23.84592628479004
773 23.319114685058594
774 24.872697830200195
775 22.343852996826172
776 24.08487892150879
777 23.154579162597656
778 23.773000717163086
779 23.656341552734375
780 24.53502655029297
781 24.904085159301758
782 24.017131805419922
783 23.79703712463379
784 21.216657638549805
785 22.90578269958496
786 23.77842140197754
787 26.34911346435547
788 24.084518432617188
789 23.572586059570312
790 24.246726989746094
791 23.52903175354004
792 23.12734603881836
793 23.878562927246094
794 23.0089111328125
795 23.570030212402344
796 23.822683334350586
797 24.328397750854492
798 25.026819229125977
79

124 23.740509033203125
125 21.38019561767578
126 21.969240188598633
127 24.098466873168945
128 24.194963455200195
129 22.734848022460938
130 23.786697387695312
131 22.64082908630371
132 23.43085479736328
133 23.608428955078125
134 24.73499870300293
135 24.875049591064453
136 23.9473819732666
137 24.198345184326172
138 24.396713256835938
139 23.393020629882812
140 23.71965789794922
141 25.27553367614746
142 23.16086196899414
143 25.51481819152832
144 22.667224884033203
145 23.630475997924805
146 23.60556411743164
147 23.45244598388672
148 24.051692962646484
149 25.318471908569336
150 23.393341064453125
151 21.493335723876953
152 24.694419860839844
153 24.32221794128418
154 21.934497833251953
155 24.709819793701172
156 23.336151123046875
157 24.55181312561035
158 23.579132080078125
159 22.617557525634766
160 22.822193145751953
161 24.414344787597656
162 22.09501838684082
163 23.51759147644043
164 23.641157150268555
165 24.446359634399414
166 24.84103012084961
167 24.959346771240234
168 2

490 24.35306167602539
491 23.400869369506836
492 23.72490882873535
493 25.043251037597656
494 22.124635696411133
495 23.984872817993164
496 24.23866081237793
497 26.93173599243164
498 24.0521183013916
499 25.020408630371094
500 23.7542724609375
501 24.272293090820312
502 25.829946517944336
503 23.534393310546875
504 22.646604537963867
505 22.631816864013672
506 22.969545364379883
507 23.134389877319336
508 23.81220245361328
509 23.917797088623047
510 23.542715072631836
511 23.36107635498047
512 22.684450149536133
513 25.094837188720703
514 21.587949752807617
515 24.554256439208984
516 25.564525604248047
517 22.421627044677734
518 24.797407150268555
519 23.165542602539062
520 22.815269470214844
521 24.546056747436523
522 23.721776962280273
523 21.496706008911133
524 23.66866683959961
525 23.523042678833008
526 24.386249542236328
527 24.443464279174805
528 22.976791381835938
529 23.60650062561035
530 26.667131423950195
531 25.070148468017578
532 24.59031867980957
533 24.409032821655273
5

855 23.854387283325195
856 26.202909469604492
857 23.25408363342285
858 23.583253860473633
859 23.41228675842285
860 24.507226943969727
861 23.755573272705078
862 23.48463249206543
863 24.379634857177734
864 23.551366806030273
865 26.02851104736328
866 22.018552780151367
867 22.083189010620117
868 23.77334976196289
869 25.059043884277344
870 22.82516860961914
871 23.731761932373047
872 22.95470428466797
873 24.69215202331543
874 24.928470611572266
875 22.678482055664062
876 21.353565216064453
877 23.751258850097656
878 23.11243438720703
879 24.4842472076416
880 24.359411239624023
881 23.679180145263672
882 22.526540756225586
883 22.59945297241211
884 21.8439998626709
885 23.823781967163086
886 23.456026077270508
887 21.530738830566406
888 22.493024826049805
889 21.320886611938477
890 22.685062408447266
891 22.043781280517578
892 24.09496307373047
893 23.203449249267578
894 23.37211799621582
895 24.885326385498047
896 23.080657958984375
897 22.654420852661133
898 22.05522918701172
899 2